In [1]:
from pymongo import MongoClient

In [2]:
host_ip = !gcloud compute instances describe datascience-main --zone=us-east1-b | grep natIP | awk -F': ' '{print $2}'

In [5]:
client = MongoClient(host='mongodb://{}'.format(host_ip[0]), port=27017)
db = client.financials
requests_coll = db.requests

In [6]:
requests_coll.find().count()

/Users/ray/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


148641